Fork of https://gist.github.com/flatline/838202
- Modified EightClass implementation, added initialization with given initial state
- Added Hamming heurisic implementation
- Added function to check solvability

In [3]:
from puzzle import h_manhattan, h_hamming, is_solvable, EightPuzzle

In [6]:
import time, timeit

# Solvability

Some description, time complexity etc.

## Unsolvable

In [4]:
# TODO: time to check if it is solvable when its not 
initial_state = [
    [8, 1, 2],
    [0, 4, 3],
    [7, 6, 5]
]
is_solvable(initial_state)

False

## Solvable

In [ ]:
# like above 

# Heuristics

In [8]:
results = []
def results_print():
    for result in results:
        print(result[0], '\n', result[1], "TIME", result[2], "CPU", result[3]) # rozbic na wiecej printów

## Manhattan

In [9]:
initial_state = [
    [1, 5, 2],
    [4, 0, 3],
    [7, 8, 6]
]

solvable = is_solvable(initial_state)
if solvable:
    p = EightPuzzle(initial_state)
    start = time.time()
    clock_start = timeit.default_timer()
    path, count = p.solve(h_manhattan)
    elapsed_time = time.time() - start
    elapsed_clock = timeit.default_timer() - clock_start
    print("Solved with ", h_manhattan, ': ', count, "states", "time", elapsed_time)
    results.append(("Mannhattan", p, elapsed_time, elapsed_clock))

Solved with  <function h_manhattan at 0x0000027A079D2E18> :  5 states time 0.0


In [10]:
results_print()

Mannhattan 
 1 5 2
4 0 3
7 8 6
 TIME 0.0 CPU 0.00020459999999999923


## Hamming

In [11]:
initial_state = [
    [1, 5, 2],
    [4, 0, 3],
    [7, 8, 6]
]

solvable = is_solvable(initial_state)
if solvable:
    p = EightPuzzle(initial_state)
    start = time.time()
    clock_start = timeit.default_timer()
    path, count = p.solve(h_hamming)
    elapsed_time = time.time() - start
    elapsed_clock = timeit.default_timer() - clock_start
    print("Solved with ", h_hamming, ': ', count, "states", "time", elapsed_time)
    results.append(("Hamming", p, elapsed_time, elapsed_clock))

Solved with  <function h_hamming at 0x0000027A079D2840> :  16 states time 0.0


In [12]:
results_print()

Mannhattan 
 1 5 2
4 0 3
7 8 6
 TIME 0.0 CPU 0.00020459999999999923
Hamming 
 1 5 2
4 0 3
7 8 6
 TIME 0.0 CPU 0.0006327000000005967


## Comparsion

## Summary